In [ ]:
import os
import json
import random
import sys
from tqdm import tqdm

# Add the parent directory (project_folder) to the system path
# This allows Python to find main_script
current_dir = os.path.dirname(__file__)
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
sys.path.append(parent_dir)

from coarse_grain_model import GPT2WithSlidingWindow
from calculate_refusal_delta import load_classifier, is_refusal, count_refusals

In [ ]:
model_path="./models/gpt2"
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

classifier_model, classifier_tokenizer = load_model(device,"./models/llm-refusal-classifier")

gen_model, gen_tokenizer= load_model(device,model_path)

print(f"Loading masked generative model using: {model_path}")
model_path = "./models/gpt2" 
WINDOW_SIZE = 5  # Use a small window for easy verification

# 1. Load tokenizer and model
tokenizer2 = GPT2Tokenizer.from_pretrained(model_path)
model2 = GPT2WithSlidingWindow.from_pretrained(model_path,WINDOW_SIZE)
model2.eval() # Set model to evaluation mode

# 3. Load Safety Datasets
safety_data_path = "./data/safety_evaluation_prompts.json"
print(f"Loading safety data from: {safety_data_path}")
try:
    with open(safety_data_path, 'r', encoding='utf-8') as f:
        safety_data = json.load(f)
    harmless_prompts = safety_data['harmless']
    jailbreak_prompts = safety_data['jailbreak']
except FileNotFoundError:
    print(f"❌ Error: Safety data not found at {safety_data_path}. Please run download_data.py first.")

In [ ]:
random_number = random.randint(0, 1000)
prompt=jailbreak_prompts[random_number]# harmless_prompts[random_number]
print("GPT2 base model being used")
print("input=",prompt)
inputs = gen_tokenizer(prompt, return_tensors="pt").to(device)
        
with torch.no_grad():
    outputs = gen_model.generate(
        **inputs,
        max_new_tokens=50,
        # temperature=0.7,
        # top_p=0.95,
        # do_sample=True,
        pad_token_id=gen_tokenizer.eos_token_id
    )

full_response = gen_tokenizer.decode(outputs[0], skip_special_tokens=True)
generated_part = full_response[len(prompt):].strip()
print("llm output=",generated_part)
answer=is_refusal(generated_part, classifier_model, classifier_tokenizer, device)
print("Refusal=",answer)

In [ ]:
random_number = random.randint(0, 1000)
prompt=jailbreak_prompts[random_number]# harmless_prompts[random_number]
print("GPT2 masked model being used")
print("input=",prompt)
inputs = tokenizer2(prompt, return_tensors="pt").to(device)
        
with torch.no_grad():
    outputs = model2.generate(
        **inputs,
        max_new_tokens=50,
        # temperature=0.7,
        # top_p=0.95,
        # do_sample=True,
        pad_token_id=gen_tokenizer.eos_token_id
    )

full_response = tokenizer2.decode(outputs[0], skip_special_tokens=True)
generated_part = full_response[len(prompt):].strip()
print("llm output=",generated_part)
answer=is_refusal(generated_part, classifier_model, classifier_tokenizer, device)
print("Refusal=",answer)